# HCAN 361 Final Project: Spring 2021

## Physical Access to Imaging Equipment in Hospitals Among Women Aged 40 Years and Older in the Southeastern US : Implications on Breast Cancer Screening

In [1]:
use [HCAN361_FinalProj]

--step 1: select the ID's of major imaging 
--drop table #key_ID
SELECT A.ID
      ,cast(A.LAT as float) as LAT
	  ,cast(A.LONG as float) as LONG
	  ,A.MNAME
      ,C.County_Name,C.State_Name,C.HRSA_RURAL_DESC,('Z'+C.STATEFP+C.COUNTYFP) as [County_GEOID_Z]
	  ,case when C.[Median_ADI_NatRank]<=25 then '02: Upper Class'
	   when (C.[Median_ADI_NatRank]>25 and C.[Median_ADI_NatRank]<=75) then '01: Middle Class'
	   when C.[Median_ADI_NatRank]>75 then '00: Lower Class'
	   else '99: Not Ranked' end as [ADI]
into #key_ID
FROM 
dbo.AS19DEM as A
left join
[dbo].[Provider_Census_Tract] as B
	on A.ID=B.Provider_ID
left join 
[dbo].[US_Census_Tract_Charac] as C
	on B.GEOID_Data=C.GEOID_Data
where A.ID in
(
select ID
from [dbo].[AS19UTIL2]
where SMRI=1
or
SMRIOC=1
or
SMRIOF=1
) and C.County_Name is not null
order by C.State_Name;
--1,642 of these hospitals


select top 10 * from #key_ID;

select distinct *
into HCAN361_25.dbo.Locs_Group6
from #key_ID;

alter table HCAN361_25.dbo.Locs_Group6
alter column ID varchar(30) not null;

alter table HCAN361_25.dbo.Locs_Group6
add constraint pk_imag_Grp6 primary key(ID);


(1642 rows affected)

(10 rows affected)

(1642 rows affected)

Total execution time: 00:00:00.549

ID,LAT,LONG,MNAME,County_Name,State_Name,HRSA_RURAL_DESC,County_GEOID_Z,ADI
6110003,43.906,-69.8904,Mid Coast Hospital,Cumberland,Maine,01: Rural,Z23005,01: Middle Class
6110008,43.6514,-70.2649,Northern Light Mercy Hospital,Cumberland,Maine,00: Urban,Z23005,01: Middle Class
6110050,44.8074,-68.7526,Northern Light Eastern Maine Medical Center,Penobscot,Maine,00: Urban,Z23019,01: Middle Class
6110090,44.385,-68.2022,Mount Desert Island Hospital,Hancock,Maine,01: Rural,Z23009,02: Upper Class
6110120,44.4122,-68.9939,Waldo County General Hospital,Waldo,Maine,01: Rural,Z23027,01: Middle Class
6110290,47.2656,-68.592,Northern Maine Medical Center,Aroostook,Maine,01: Rural,Z23003,01: Middle Class
6110310,45.4588,-69.6111,Northern Light CA Dean Hospital,Piscataquis,Maine,01: Rural,Z23021,01: Middle Class
6110367,44.0259,-69.53,LincolnHealth,Lincoln,Maine,01: Rural,Z23015,01: Middle Class
6110378,43.4776,-70.4904,Southern Maine Health Care - Biddeford Medical Center,York,Maine,00: Urban,Z23031,01: Middle Class
6110386,44.7136,-67.4771,Down East Community Hospital,Washington,Maine,01: Rural,Z23029,01: Middle Class


In [2]:

--step 2: find target population of interest for these services
--women over 40 (B01001e38) to women 85+ (B01001e49) 
--drop table #pop
select A.GEOID_Data, 'Z'+B.STATEFP+B.COUNTYFP as [County_GEOID_Z],B.State_Name,B.County_Name,
	B01001e38+B01001e39+B01001e40+B01001e41+B01001e42+B01001e43+B01001e44+B01001e45+B01001e46+B01001e47+B01001e48+B01001e49 as [womenover40]
into #pop
from 
[dbo].[Census_ACS5Yr_2018] as A
left join 
[dbo].[US_Census_Tract_Charac] as B
	on A.GEOID_Data=B.GEOID_Data
order by GEOID_Data;
--74,001 rows 

select top 10 *
from #pop;

(74001 rows affected)

(10 rows affected)

Total execution time: 00:00:00.295

GEOID_Data,County_GEOID_Z,State_Name,County_Name,womenover40
14000US06037206050,Z06037,California,Los Angeles,729.00000000
14000US06037206200,Z06037,California,Los Angeles,1075.00000000
14000US06037208301,Z06037,California,Los Angeles,229.00000000
14000US06037208302,Z06037,California,Los Angeles,1057.00000000
14000US06037208720,Z06037,California,Los Angeles,762.00000000
14000US06037208802,Z06037,California,Los Angeles,718.00000000
14000US06037208903,Z06037,California,Los Angeles,607.00000000
14000US06037208904,Z06037,California,Los Angeles,748.00000000
14000US06037209401,Z06037,California,Los Angeles,911.00000000
14000US06037211310,Z06037,California,Los Angeles,676.00000000


In [3]:

--Step 3: Urban vs Rural Areas in Southern Region
--drop table #south_charac
select A.[GEOID_Data],A.County_Name,A.State_Name,A.HRSA_RURAL_DESC
      ,('Z'+A.STATEFP+A.COUNTYFP) as [County_GEOID_Z]
	  ,case when A.[Median_ADI_NatRank]<=25 then '02: Upper Class'
		    when (A.[Median_ADI_NatRank]>25 and A.[Median_ADI_NatRank]<=75) then '01: Middle Class'
		    when A.[Median_ADI_NatRank]>75 then '00: Lower Class'
		    else '99: Not Ranked' end as [ADI]
	  ,B.womenover40
into #south_charac
from 
[dbo].[US_Census_Tract_Charac] A
left join
#pop B
	on A.GEOID_Data=B.GEOID_Data
where 
(A.[State_Name] like '%Tennessee%'
or A.[State_Name] like '%North Carolina%'
or A.[State_Name] like '%South Carolina%'
or A.[State_Name] like '%Georgia%'
or A.[State_Name] like '%Florida%'
or A.[State_Name] like '%Alabama%'
or A.[State_Name] like '%Mississippi%'
or A.[State_Name] like '%Louisiana%'
or A.[State_Name] like '%Missouri%'
or A.[State_Name] like '%Arkansas%') 
--16,081 rows

select top 10 * from #south_charac

(16081 rows affected)

(10 rows affected)

Total execution time: 00:00:00.430

GEOID_Data,County_Name,State_Name,HRSA_RURAL_DESC,County_GEOID_Z,ADI,womenover40
14000US12086009044,Miami-Dade,Florida,00: Urban,Z12086,01: Middle Class,579.00000000
14000US12086009047,Miami-Dade,Florida,00: Urban,Z12086,01: Middle Class,677.00000000
14000US12086009100,Miami-Dade,Florida,00: Urban,Z12086,01: Middle Class,1847.00000000
14000US12086009901,Miami-Dade,Florida,00: Urban,Z12086,01: Middle Class,2216.00000000
14000US12086009904,Miami-Dade,Florida,00: Urban,Z12086,01: Middle Class,631.00000000
14000US13037950200,Calhoun,Georgia,01: Rural,Z13037,00: Lower Class,877.00000000
14000US13039010403,Camden,Georgia,01: Rural,Z13039,01: Middle Class,1133.00000000
14000US13039010500,Camden,Georgia,01: Rural,Z13039,00: Lower Class,129.00000000
14000US13039010601,Camden,Georgia,01: Rural,Z13039,01: Middle Class,1815.00000000
14000US13039010602,Camden,Georgia,01: Rural,Z13039,01: Middle Class,270.00000000


In [4]:

--Step 4: Matrix for Map
--drop table #key_table
select GEOID_Data, count(distinct Provider_ID) as [hospitals_win_60min], min([Total_TravelTime]) as [closest_majorimaging]
into #key_table
from 
(select A.*, B.womenover40, C.[State_Name]
from [dbo].[Travel_Matrix] as A
left join #pop as B on A.GEOID_Data=B.GEOID_Data
left join #south_charac as C on A.GEOID_Data=C.GEOID_Data
where Provider_ID in (select ID from #key_ID)
and Total_TravelTime<=60
and C.[State_Name] is not null ) as A
group by GEOID_Data
order by GEOID_Data 
--14,617 rows 

select top 10 *
from #key_table


(14617 rows affected)

(10 rows affected)

Total execution time: 00:00:03.381

GEOID_Data,hospitals_win_60min,closest_majorimaging
14000US12073002005,1,50.60655342846121
14000US45069960100,1,28.796205250654857
14000US37071030901,7,34.24175098595023
14000US29510114101,20,10.382194325306216
14000US28047003204,3,31.97364670757619
14000US12099004300,8,13.279148879739326
14000US29183310501,22,3.565826481055893
14000US37001020902,10,12.192948417782109
14000US28059040102,1,35.47328252085533
14000US12086018300,12,2.789609730085362


In [5]:

--Step 5: Final Table for Mapping
--drop table #final_table
select a.GEOID_Data,
	   a.State_Name,
	   a.County_Name,
	   a.County_GEOID_Z,
	   a.ADI,
	   a.HRSA_RURAL_DESC,
	   a.womenover40,
	   isnull(b.hospitals_win_60min,0) as hospitals_win_60min,
	   isnull(b.closest_majorimaging,361) as closest_majorimaging
into #final_table 
from #south_charac as A 
left join #key_table as B on A.GEOID_Data=B.GEOID_Data
where [State_Name] is not null 
and 
[ADI] is not null
and 
[HRSA_RURAL_DESC] is not null
order by GEOID_Data;
--16,081 rows

--select count(GEOid_Data) as total
--from [dbo].[US_Census_Tract_Charac] A
--where
--(A.[State_Name] like '%Tennessee%'
--or A.[State_Name] like '%North Carolina%'
--or A.[State_Name] like '%South Carolina%'
--or A.[State_Name] like '%Georgia%'
--or A.[State_Name] like '%Florida%'
--or A.[State_Name] like '%Alabama%'
--or A.[State_Name] like '%Mississippi%'
--or A.[State_Name] like '%Louisiana%'
--or A.[State_Name] like '%Missouri%'
--or A.[State_Name] like '%Arkansas%')
----16,081 rows...The numbers match up

select top 10 * from #final_table;

select distinct *
into HCAN361_25.dbo.TravelMap_Group6
from #final_table;

alter table HCAN361_25.dbo.TravelMap_Group6
alter column GEOID_Data varchar(30) not null;

alter table HCAN361_25.dbo.TravelMap_Group6
add constraint pk_trvl_Grp6 primary key(GEOID_Data);

(16081 rows affected)

(10 rows affected)

(16081 rows affected)

Total execution time: 00:00:00.661

GEOID_Data,State_Name,County_Name,County_GEOID_Z,ADI,HRSA_RURAL_DESC,womenover40,hospitals_win_60min,closest_majorimaging
14000US12101032110,Florida,Pasco,Z12101,01: Middle Class,00: Urban,938.00000000,5,22.3191836671055
14000US12101032111,Florida,Pasco,Z12101,01: Middle Class,00: Urban,945.00000000,5,27.593855732419893
14000US12101032112,Florida,Pasco,Z12101,01: Middle Class,00: Urban,955.00000000,5,22.713729194525293
14000US12101032113,Florida,Pasco,Z12101,02: Upper Class,00: Urban,1046.00000000,5,26.221368698988375
14000US12101032803,Florida,Pasco,Z12101,01: Middle Class,00: Urban,652.00000000,2,41.59762583546482
14000US12101032804,Florida,Pasco,Z12101,00: Lower Class,00: Urban,1121.00000000,2,38.16925349011617
14000US12101032901,Florida,Pasco,Z12101,01: Middle Class,00: Urban,1334.00000000,2,37.52052113488137
14000US12101032903,Florida,Pasco,Z12101,00: Lower Class,00: Urban,1514.00000000,2,34.872691768501014
14000US12101033011,Florida,Pasco,Z12101,01: Middle Class,00: Urban,617.00000000,2,32.1293152433225
14000US12101990000,Florida,Pasco,Z12101,99: Not Ranked,00: Urban,0.00000000,5,30.8050520209651


In [9]:
--Step 6: County-Level rate of hospital radiologists per 5000 women aged 40 years and older
--FTRAD-->Full-time radiology technicians-->as19util1

select distinct A.*,(FTRAD/( order by Radiologists_per_5K_Women descwomenover40*1.00))*5000 as Radiologists_per_5K_Women
into #county
from
(
select 
 A.County_GEOID_Z,A.State_Name,A.County_Name,isnull(B.FTRAD,0) as FTRAD
,isnull(sum(A.womenover40),0) as womenover40
from 
#south_charac as A
left join
(select A.County_GEOID_Z, sum(B.FTRAD) as FTRAD
from #key_ID as A left join dbo.as19util1 as B on A.ID=B.ID
where County_GEOID_Z in (select County_GEOID_Z from #south_charac)
group by A.County_GEOID_Z
)as B
	on A.County_GEOID_Z=B.County_GEOID_Z
group by A.County_GEOID_Z,A.State_Name,A.County_Name,B.FTRAD
)A


select top 10 * from #county;

select distinct *
into HCAN361_25.dbo.County_Group6
from #county;

alter table HCAN361_25.dbo.County_Group6
alter column County_GEOID_Z varchar(30) not null;

alter table HCAN361_25.dbo.County_Group6
add constraint pk_county_Grp6 primary key(County_GEOID_Z);

(870 rows affected)

(10 rows affected)

(870 rows affected)

Total execution time: 00:00:00.308

County_GEOID_Z,State_Name,County_Name,FTRAD,womenover40,Radiologists_per_5K_Women
Z12093,Florida,Okeechobee,113,9952.00000000,56.77250803858521
Z37135,North Carolina,Orange,284,33789.00000000,42.0255112610613
Z13277,Georgia,Tift,57,9866.00000000,28.887086965335495
Z37063,North Carolina,Durham,371,70857.00000000,26.17948826509731
Z13059,Georgia,Clarke,108,22665.00000000,23.825281270681668
Z28071,Mississippi,Lafayette,46,10728.00000000,21.439224459358687
Z13227,Georgia,Pickens,37,9195.00000000,20.11963023382273
Z37139,North Carolina,Pasquotank,38,10273.00000000,18.49508420130439
Z13275,Georgia,Thomas,45,12277.00000000,18.32695283864136
Z37165,North Carolina,Scotland,31,9065.00000000,17.09873138444567


In [7]:

--Table 1: Median Number of Facilites with Major Imaging Equipment 
select *
from
(
select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hospitals_win_60min)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then hospitals_win_60min end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then hospitals_win_60min end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then hospitals_win_60min end) OVER () AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then hospitals_win_60min end) OVER () AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then hospitals_win_60min end) OVER () AS [Low SES]
from #final_table

--below code concatenates (or stacks tables on top of each other)
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hospitals_win_60min)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS [Low SES]
from #final_table) as A
order by State_Name;
--11 rows


(11 rows affected)

Total execution time: 00:00:00.991

State_Name,Statewide,Urban,Rural,High SES,Middle SES,Low SES
Alabama,3,3,2,5,3,2
Arkansas,4,8,2,8,7,3
Florida,5,5,2,6,5,5
Georgia,5,12,2,14,5,3
Louisiana,5,7,1,10,6,2
Mississippi,2,3,2,3,2,2
Missouri,17,20,2,20,15,4
North Carolina,5,6,2,6,5,4
South Carolina,0,0,0,0,0,0
Tennessee,3,3,1,3,3,3


In [8]:

--Table 2: Median Number of Closest Facilities with Major Imaging 
select *
from
(select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY closest_majorimaging)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then closest_majorimaging end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then closest_majorimaging end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then closest_majorimaging end) OVER () AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then closest_majorimaging end) OVER () AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then closest_majorimaging end) OVER () AS [Low SES]
from #final_table

--below code concatenates (or stacks tables on top of each other)
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY closest_majorimaging)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then closest_majorimaging end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then closest_majorimaging end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then closest_majorimaging end) OVER (PARTITION BY State_Name) AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then closest_majorimaging end) OVER (PARTITION BY State_Name) AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then closest_majorimaging end) OVER (PARTITION BY State_Name) AS [Low SES]
from #final_table) as A
order by State_Name;
--11 rows


(11 rows affected)

Total execution time: 00:00:01.007

State_Name,Statewide,Urban,Rural,High SES,Middle SES,Low SES
Alabama,24.304102779691323,19.787232880320644,41.96088578943745,17.170890159272567,24.294236504728765,28.42178137996215
Arkansas,20.23434180285927,12.754125541734645,37.095745775073155,11.32962954471392,19.21467863500221,26.226229523090808
Florida,18.54695987519991,18.01853092554183,42.57679927887849,16.882593084381533,18.68690455863151,18.76961960347763
Georgia,20.458586608337725,17.021337993438316,38.64439250763651,15.500652595476616,21.23179958106383,23.56686181867076
Louisiana,14.993817476035444,12.149860277812031,44.09313939281704,11.035403767165407,15.430830911564312,18.61540956414723
Mississippi,28.980103174837186,24.96748687654574,31.084784862317495,20.398877761215708,26.883407483592435,30.93330382640211
Missouri,14.35243769416516,11.8280221581469,32.57334540881746,11.672111706076969,15.266326279470997,14.049971160092745
North Carolina,19.883266148686047,16.760644353433893,34.885028459403074,17.136422041527172,20.392470193063517,22.445009231939537
South Carolina,361,361,361,361,361,361
Tennessee,23.45568307172841,18.74621706826916,41.20515769976943,21.825903899861174,25.04506095973368,18.005515249947514
